In [49]:
from dotenv import load_dotenv
import matplotlib.pyplot as plt

from eth_rpc import PrivateKeyWallet
from emp_orderly.utils import from_address
from emp_orderly import (
    Strategy, EmpOrderly,
    crossover, EMA,
    EmpyrealOrderlySDK,
)
from emp_orderly_types import PerpetualAssetType, Interval

load_dotenv()

True

In [50]:
wallet = PrivateKeyWallet.create_new()
orderly_id = from_address(wallet.address)

sdk = EmpyrealOrderlySDK(pvt_hex=wallet.private_key, account_id=orderly_id, is_testnet=True)

In [51]:
print(wallet.address)

0x5AbcfBEe2B501eE2452731C45cA3294A741F10fE


In [52]:
print(await wallet.balance())

0


In [53]:
from typing import Union
import pandas_ta as ta
import pandas as pd


class EmaCross(Strategy):
    """Exponential Moving Average Death/Golden Cross Strategy.
    
    order_size: The amount of order that can be 
        made at a time.
    parameters: The parameters
    """

    # Define the strategy parameters
    parameters: dict[str, Union[int, float]] = {
        "fast_ema_period": 5,
        "slow_ema_period": 20,
        "rsi_period": 14,
        "rsi_overbought": 70,
        "rsi_oversold": 30,
        "risk_percentage": 0.02,  # 2% risk per trade
        "reward_risk_ratio": 2,   # Reward to risk ratio
    }

    order_size: float = 0.999
    
    def init(self):
        close = self.data.close
        self.fast_ema = self.I(EMA, close, self.parameters["fast_ema_period"])
        self.slow_ema = self.I(EMA, close, self.parameters["slow_ema_period"])
        self.current_price = self.data.df.close.iloc[-1]

    def next(self):
        rsi = ta.rsi(self.data.df.close, self.parameters["rsi_period"])
        current_rsi = rsi.iloc[-1]
        if (
            crossover(self.fast_ema, self.slow_ema)
            and
            self.parameters["rsi_oversold"] < current_rsi < self.parameters["rsi_overbought"]
        ):
            stop_loss_price = self.current_price * (1 - self.parameters["risk_percentage"])
            take_profit_price = self.current_price * (1 + self.parameters["risk_percentage"] * self.parameters["reward_risk_ratio"])
            limit_price = stop_loss_price + 0.001
            self.buy(size=self.order_size, sl=stop_loss_price, tp=take_profit_price, limit=limit_price)
        elif (
            crossover(self.slow_ema, self.fast_ema)
            and
            self.parameters["rsi_oversold"] < current_rsi < self.parameters["rsi_overbought"]
        ):
            stop_loss_price = self.current_price * (1 + self.parameters["risk_percentage"])
            take_profit_price = self.current_price * (1 - self.parameters["risk_percentage"] * self.parameters["reward_risk_ratio"])
            limit_price = stop_loss_price - 0.001
            self.sell(size=self.order_size, sl=stop_loss_price, tp=take_profit_price, limit=limit_price)
        # if (
        #     crossover(self.fast_ema, self.slow_ema)
        #     and
        #     self.parameters["rsi_oversold"] < current_rsi < self.parameters["rsi_overbought"]
        # ):
        #     self.position.close()
        #     self.buy(size=self.order_size)
        # elif (
        #     crossover(self.slow_ema, self.fast_ema)
        #     and
        #     self.parameters["rsi_oversold"] < current_rsi < self.parameters["rsi_overbought"]
        # ):
        #     self.position.close()
        #     self.sell(size=self.order_size)


tester = EmpOrderly(
    cash=100000,
    commission=.000001,
    exclusive_orders=True,
    sdk=sdk,
)

# load strategy and data
tester.set_strategy(EmaCross)

# # BONK
# await tester.load_data(
#     lookback=14,
#     interval=Interval.fifteen_minute,
#     asset=PerpetualAssetType.BONK,
# )
# # backtest
# BONK = tester.backtest()

# # REZ
# await tester.load_data(
#     lookback=14,
#     interval=Interval.fifteen_minute,
#     asset=PerpetualAssetType.REZ,
# )
# # backtest
# REZ = tester.backtest()

# # ONDO
# await tester.load_data(
#     lookback=14,
#     interval=Interval.fifteen_minute,
#     asset=PerpetualAssetType.ONDO,
# )
# # backtest
# ONDO = tester.backtest()

# # OP
# await tester.load_data(
#     lookback=14,
#     interval=Interval.fifteen_minute,
#     asset=PerpetualAssetType.OP,
# )
# # backtest
# OP = tester.backtest()

# # ORDI
# await tester.load_data(
#     lookback=14,
#     interval=Interval.fifteen_minute,
#     asset=PerpetualAssetType.ORDI,
# )
# # backtest
# ORDI = tester.backtest()

# # WOO
# await tester.load_data(
#     lookback=14,
#     interval=Interval.fifteen_minute,
#     asset=PerpetualAssetType.WOO,
# )
# # backtest
# WOO = tester.backtest()

# # ETH
# await tester.load_data(
#     lookback=14,
#     interval=Interval.fifteen_minute,
#     asset=PerpetualAssetType.ETH,
# )
# # backtest
# ETH = tester.backtest()

# # OMNI
# await tester.load_data(
#     lookback=14,
#     interval=Interval.fifteen_minute,
#     asset=PerpetualAssetType.OMNI,
# )
# # backtest
# OMNI = tester.backtest()

# SEI
await tester.load_data(
    lookback=14,
    interval=Interval.fifteen_minute,
    asset=PerpetualAssetType.SEI,
)
# backtest
SEI = tester.backtest()

results = pd.DataFrame([
    # BONK,
    # REZ,
    # ONDO,
    # OP,
    # ORDI,
    # WOO,
    # ETH,
    # OMNI,
    SEI,
])
results

/home/ademola/.cache/pypoetry/virtualenvs/balo-orderly-submission-Pfl10bGM-py3.12/lib/python3.12/site-packages/emp_orderly/backtest/backtesting.py:942: UserWarning: (2024-09-30 07:45:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119
  warnings.warn(
/home/ademola/.cache/pypoetry/virtualenvs/balo-orderly-submission-Pfl10bGM-py3.12/lib/python3.12/site-packages/emp_orderly/backtest/backtesting.py:942: UserWarning: (2024-09-30 09:15:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) pri

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],...,Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,Kelly Criterion,_strategy,_equity_curve,_trades
0,2024-09-28 14:15:00,2024-10-12 14:00:00,13 days 23:45:00,7.291667,105897.039663,109450.816939,5.89704,9.647113,281.496255,162.249228,...,0.143483,0 days 03:15:00,0 days 00:22:00,1.545283,0.150148,0.771654,0.159597,EmaCross,Equity DrawdownPc...,Size EntryBar ExitBar EntryPrice Exi...


In [54]:
# plot
tester.plot(show_price_data=False)

GridPlot(id='p3398', ...)

In [55]:
SEI

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/ademola/snap/code/common/.cache/gio-modules/libgiolibproxy.so


Start                     2024-09-28 14:15:00
End                       2024-10-12 14:00:00
Duration                     13 days 23:45:00
Exposure Time [%]                    7.291667
Equity Final [$]                105897.039663
Equity Peak [$]                 109450.816939
Return [%]                            5.89704
Buy & Hold Return [%]                9.647113
Return (Ann.) [%]                  281.496255
Volatility (Ann.) [%]              162.249228
Sharpe Ratio                         1.734962
Sortino Ratio                       14.651358
Calmar Ratio                        86.696472
Max. Drawdown [%]                   -3.246917
Avg. Drawdown [%]                   -1.484271
Max. Drawdown Duration        4 days 11:30:00
Avg. Drawdown Duration        1 days 07:30:00
# Trades                                   40
Win Rate [%]                             47.5
Best Trade [%]                       5.896517
Worst Trade [%]                     -1.315302
Avg. Trade [%]                    

Opening in existing browser session.
